### Mg (10m10) Onsurface Adsorption Surface Pourbaix Diagram Notebook 

1. Construct the on-surface adsorption surfaces
   
* Import required packages for the structure construction

In [1]:
import sys
# need to determine the path for the surface pourbaix-helper package
sys.path.append("/Users/libingxin/Project/SurfacePourbaix/SurfacePourbaixHelper/")

In [2]:
from SlabBuilder import SlabBuilder
from Mg10m10SiteFinder import Mg_10m10_AdsorptionFinder as SiteFinder
from Mg10m10AdsorptionHelper import Mg_10m10_AdsorptionAdder as AdsorpAdder
from StructureOutput import StructureOutput
import numpy as np
import copy as cp
import os
import subprocess

2. Build up the Mg hcp bulk structure and the (10m10) surface with relaxed VASP parameters

In [3]:
a_vasp = 3.1976346746068987
c_vasp = 5.1651843977024186

lattice = [a_vasp, a_vasp, c_vasp]

Mg = SlabBuilder('Mg','hcp',lattice)

# Constructing the bulk hcp Mg structure
Mg.BulkStructureBuilder()

# Constructing the Mg (10m10) surface with 8 layers and 20 Angstrom vacuum length
supercell = np.array([2,2,1])
Mg.SlabBuilder([1,0,0],8,10,supercell)
Mg10m10 = Mg.supercellslab

# Determine all possible adsorption sites
# Check the position of different sites with Mg_10m10_af.[The Sites You Want:HW,OT,SB,SB and OCTa/b, TetraIa/b, TetraIIa/b]
adsorption = SiteFinder(Mg10m10)
adsorption.LayerFinder()

adsorption.OnSurfAdsorptionFinder()
adsorption.SubSurfAdsorptionFinder(a_vasp,c_vasp)

In [4]:
all_onsurf_site = dict([("HW",adsorption.HW), ("LB",adsorption.LB), ("OT",adsorption.OT), ("SB",adsorption.SB)])
all_subsurf_site = dict([("OCTa",adsorption.OCTa), ("OCTb",adsorption.OCTb),
                         ("TetraIa",adsorption.TetraIa), ("TetraIb",adsorption.TetraIb), 
                         ("TetraIIa",adsorption.TetraIIa), ("TetraIIb",adsorption.TetraIIb)])

3. Generate all onsurface adsorption structures and corresponding directories
* Generating pure adsorption surface structures 

In [5]:
adsorbate = ["O","OH"]
sites = list(all_onsurf_site.keys())

In [6]:
for site in sites:
    for coverage in range(1,5):
        if coverage != 2:
            for ads in adsorbate:
                dir = 'Mg_10m10_' + str(coverage) + str(ads) + '_' + str(site) 
                os.mkdir(dir)
                os.system('cp INCAR KPOINTS %s' % dir)
                os.chdir(dir)

                slab = AdsorpAdder(Mg10m10,all_onsurf_site[site])
                slab.PureAdsorption(coverage/4,ads)

                writer = StructureOutput("POSCAR","vasp",slab.slab_Pure)
                writer.StructureWriter()

                os.chdir("..")

        if coverage == 2:
            for ads in adsorbate:
                for j in range(1,4):
                    dir = 'Mg_10m10_' + str(coverage) + str(ads) + '_' + str(site) + '_p' + str(j)
                    os.mkdir(dir)
                    os.system('cp INCAR KPOINTS %s' % dir)
                    os.chdir(dir)

                    slab = AdsorpAdder(Mg10m10,all_onsurf_site[site])
                    slab.PureAdsorption(coverage/4,ads)

                    cm = 'writer = StructureOutput("POSCAR","vasp",slab.slab_Pure2_p%i)'%j
                    exec(cm)
                    writer.StructureWriter()

                    os.chdir("..")

* Generating hybrid adsorption surface structures

In [7]:
for site in sites:
    for coverage in range(2,5):
        slab = AdsorpAdder(Mg10m10,all_onsurf_site[site])

        for n_o in range(1,coverage):
            n_oh = coverage - n_o 

            if ((n_oh == 1) and (n_o == 1)) or ((n_oh == 1) and (n_o == 2)) or ((n_oh == 2) and (n_o == 1)) or ((n_oh == 2) and (n_o == 2)):
                for j in range(1,4):
                    dir = 'Mg_10m10_' + str(n_o) + 'O_' + str(n_oh) + 'OH_' + str(site) + '_p' + str(j)
                    os.mkdir(dir)
                    os.system('cp INCAR KPOINTS %s' % dir)
                    os.chdir(dir)

                    slab.HybridAdsorption(n_o,n_oh)
                    cm = 'writer = StructureOutput("POSCAR","vasp",slab.slab_%iOH_%iO_p%i)' % (n_oh,n_o,j)
                    exec(cm)
                    writer.StructureWriter()

                    os.chdir('..')
            
            else:
                dir = 'Mg_10m10_' + str(n_o) + 'O_' + str(n_oh) + 'OH_' + str(site) 
                os.mkdir(dir)
                os.system('cp INCAR KPOINTS %s' % dir)
                os.chdir(dir)

                slab.HybridAdsorption(n_o,n_oh)
                cm = 'writer = StructureOutput("POSCAR","vasp",slab.slab_%iOH_%iO_p1)' % (n_oh,n_o)
                exec(cm)
                writer.StructureWriter()

                os.chdir('..')